# 1. Introduction

Rio Grande do Sul Public Bids.

# 2. Load Modules and Variable Environments

In [162]:
%reload_ext autoreload
%autoreload 1
# Load custom local packages
import sys
import os
sys.path.append(os.path.abspath('..'))
# Local Packages
%aimport src.features
from src.features.build_features import build_features
from src.nlp_utils.clean_text import clean_text
from src.nlp_utils.extract_ngrams import extract_ngrams
from src.data.load_data import load_data
# Standard Packages
import pandas as pd
import numpy as np
from IPython.display import HTML
import gc
from collections import Counter
# Visualization Packages
import hvplot.pandas
# Paths and Config constants
DATA_PATH = os.path.abspath('..')+os.sep+'data'
FIG_PATH = os.path.abspath('..')+os.sep+'reports'+\
            os.sep+'figures'+os.sep
YEARS = [2016, 2017, 2018, 2019]
CUSTOM_STOPWORDS = ['aquisição', 'fornecimento', 'kg', 'tk'
                    'gr', 'sc', 'conforme', 'anexo', 'edital',
                    'tipo', 'área', 'lote', 'aprox', 'porto',
                    'serviços','cor',  'rua', 'alegrers', 'item',
                    'uso', 'serviço', 'cfeespanexa',  'numero',
                     'privativa',  'vii', 'matrícula', 'objeto',
                     'amt', 'mca', 'm³h', 'amt', 'solução', 'padrão',
                     'vazão', 'objetos', 'mgml', 'termo', 'referência',
                     'registro', 'preços', 'outro', 'bens', 'ano',
                     'respectivo', 'constantes', 'humana', 'ref',
                     'etapa', 'deste', 'nao', 'aparelhos', 'protecao',
                     'preparacao']

# 3. Build Required Features

In [163]:
"""
bid_features = ['NR_LICITACAO', 'CD_ORGAO', 'NM_ORGAO', 'DS_OBJETO',
                 'TP_OBJETO', 'ANO_LICITACAO']
item_features = ['NR_LICITACAO', 'CD_ORGAO', 'DS_ITEM', 'VL_UNITARIO_HOMOLOGADO']

build_features(dataset='licitacao', data_path=DATA_PATH, cols=bid_features, values=YEARS)
print('=' * 18)
build_features(dataset='item', data_path=DATA_PATH, cols=item_features, values=YEARS)
"""

"\nbid_features = ['NR_LICITACAO', 'CD_ORGAO', 'NM_ORGAO', 'DS_OBJETO',\n                 'TP_OBJETO', 'ANO_LICITACAO']\nitem_features = ['NR_LICITACAO', 'CD_ORGAO', 'DS_ITEM', 'VL_UNITARIO_HOMOLOGADO']\n\nbuild_features(dataset='licitacao', data_path=DATA_PATH, cols=bid_features, values=YEARS)\nprint('=' * 18)\nbuild_features(dataset='item', data_path=DATA_PATH, cols=item_features, values=YEARS)\n"

# 4. Load Processed Data

In [164]:
bid, item = load_data(data_path=DATA_PATH, years=YEARS)

,CD_ORGAO,NM_ORGAO,NR_LICITACAO,ANO_LICITACAO,DS_OBJETO
0,84100,PM DE ESPERANÇA DO SUL,4.0,2016,Aquisição de merenda escolar
1,82701,CM DE XANGRI-LÁ,1.0,2016,AQUISIÇÃO DE PROTOCOLADORA E FRAGMENTADORA DE ...
2,84100,PM DE ESPERANÇA DO SUL,6.0,2016,Aquisição de materiais de limpeza para as dive...


Number of samples: 82190
 


,CD_ORGAO,NR_LICITACAO,DS_ITEM,VL_UNITARIO_HOMOLOGADO,ANO_LICITACAO
0,77200,15.0,"Pneu 1000/20 liso radial, dianteiro, 16 lonas,...",1430.0,2016
1,77200,15.0,"Pneu 12.4x24 garra baixa, 10 lonas",1258.0,2016
2,77200,15.0,Pneu 1400x24 mínimo 12 lonas,1998.0,2016


Number of samples: 3079700
 
There are no problems with Code and Organ names.
 
There are 843 Public Organs in the Dataset.


In [165]:
display(HTML('<b>Missing values for Bids'), bid.isnull().sum())
print(' ')
display(HTML('<b>Missing values for Items'), item.isnull().sum())

CD_ORGAO         0
NM_ORGAO         0
NR_LICITACAO     0
ANO_LICITACAO    0
DS_OBJETO        0
dtype: int64

CD_ORGAO                       0
NR_LICITACAO                   0
DS_ITEM                        1
VL_UNITARIO_HOMOLOGADO    604785
ANO_LICITACAO                  0
dtype: int64

# 5. Exploratory Data Analysis
## 5.1 General Analysis

In [166]:
sf = bid['ANO_LICITACAO'].value_counts()
df = pd.DataFrame({'Year': sf.index,
                    'Total': sf.values})
df.sort_values(by=['Year'], inplace=True)

fig = df.hvplot.line(x='Year', y=['Total'], grid=True,
                xticks=np.arange(2016, 2019.5, 1).tolist(),
                title='Figure 1 - Total Bids by Year (2016-2019)')

hvplot.save(fig, FIG_PATH+'fig-1.png')
fig

:Curve   [Year]   (Total)

In [167]:
df = item.groupby(['ANO_LICITACAO']).sum('VL_UNITARIO_HOMOLOGADO').\
    drop(['NR_LICITACAO'], axis=1).reset_index().sort_values(by=['ANO_LICITACAO'])

fig = df.hvplot.line(x='ANO_LICITACAO', y=['VL_UNITARIO_HOMOLOGADO'], grid=True,
                xticks=np.arange(2016, 2019.5, 1).tolist(), yformatter='%.1d',
                xlabel='Year', ylabel='Total',
                title='Figure 2 - Total Bid Costs by Year (2016-2019)')

hvplot.save(fig, FIG_PATH+'fig-2.png')
fig

:Curve   [ANO_LICITACAO]   (VL_UNITARIO_HOMOLOGADO)

In [168]:
top_n = 5
sf = bid['NM_ORGAO'].value_counts().head(top_n)
# Wil be used later
sf_cd = bid['CD_ORGAO'].value_counts().head(top_n)
top10_cd = sf_cd.index.tolist()
df = pd.DataFrame({'Public Organ': sf.index,
                    'Total': sf.values})

fig = df.hvplot.barh(x='Public Organ', y='Total', flip_yaxis=True, width=900,
                    xticks=np.arange(0, 2000, 300).tolist(), grid=True,
                    title='Figure 3 - Top 10 Public Organs by Number of Bids (2016-2019)')

hvplot.save(fig, FIG_PATH+'fig-3.png')
fig
                

:Bars   [Public Organ]   (Total)

In [169]:
sf = item.groupby(['CD_ORGAO']).sum('VL_UNITARIO_HOMOLOGADO').\
    drop(['NR_LICITACAO'], axis=1).reset_index().\
    sort_values(by=['VL_UNITARIO_HOMOLOGADO'],
                ascending=False).head(top_n)

cdo_list = sf['CD_ORGAO'].tolist()
po_names = []

for cdo in cdo_list: 
    po_list = bid.query('CD_ORGAO == @cdo')
    po_name = po_list['NM_ORGAO'].unique()
    po_names.append(po_name[0])

df = pd.DataFrame({'Public Organ': po_names,
                    'Total': sf['VL_UNITARIO_HOMOLOGADO']})

fig = df.hvplot.barh(x='Public Organ', y='Total', flip_yaxis=True,
                    grid=True, xformatter='%.1d', width=1000,
                    title=f'Figure 4 - Top {top_n} Public Organs by Total Value of Bids (2016-2019)')

hvplot.save(fig, FIG_PATH+'fig-4.png')
fig

:Bars   [Public Organ]   (Total)

In [170]:
po_top_costs = item.query('CD_ORGAO in(@cdo_list)')
top_dict = dict(zip(cdo_list, po_names))
# Top costs organs
po_top_costs['NOME_ORGAO'] = po_top_costs['CD_ORGAO'].map(top_dict)

fig = po_top_costs.hvplot.box(y='VL_UNITARIO_HOMOLOGADO', by='NOME_ORGAO',
                                width=1100, invert=True, xformatter='%.1d',
                                xlabel='Public Organ', ylabel='Cost Distribution',
                                title='Figure 5 - Item Costs Distribution from most costly Public Organs (2016-2019)')
hvplot.save(fig, FIG_PATH+'fig-5.png')
fig

:BoxWhisker   [NOME_ORGAO]   (VL_UNITARIO_HOMOLOGADO)

In [171]:
item_missing = item[item['VL_UNITARIO_HOMOLOGADO'].isnull()]
top_missing = item['CD_ORGAO'].value_counts().head(top_n)

df = pd.DataFrame({'Public Organ': top_missing.index,
                    'Total': top_missing.values})

df_list = df['Public Organ'].tolist()
df_names = []

for cdo in df_list: 
    p_list = bid.query('CD_ORGAO == @cdo')
    df_name = p_list['NM_ORGAO'].unique()
    df_names.append(df_name[0])

df['Public Organ'] = df_names

fig = df.hvplot.barh(x='Public Organ', y='Total', flip_yaxis=True,
                    grid=True, xformatter='%.1d', width=1000,
                    title=f'Figure 6 - Top {top_n} Public Organs by Item Cost Missing Values (2016-2019)')

hvplot.save(fig, FIG_PATH+'fig-6.png')
fig

:Bars   [Public Organ]   (Total)

## 5.2 Analysis on Top 5 most costly Public Organs

In [172]:

po_top_costs_bid = bid.query('CD_ORGAO in(@cdo_list)')
bid_top = clean_text(df=po_top_costs_bid, feature='DS_OBJETO', 
                    custom_stop=CUSTOM_STOPWORDS)
item_top = clean_text(df=po_top_costs, feature='DS_ITEM', 
                    custom_stop=CUSTOM_STOPWORDS)
gc.collect()
# Some wrong words to replace
bid_top['DS_OBJETO'] = bid_top['DS_OBJETO'].str.replace('pinformatica', 'informatica')
bid_top['DS_OBJETO'] = bid_top['DS_OBJETO'].str.replace('famíliasalimentacao', 'alimentacao')
bid_top['DS_OBJETO'] = bid_top['DS_OBJETO'].str.replace('famíliasequipamentos', 'equipamentos')
bid_top['DS_OBJETO'] = bid_top['DS_OBJETO'].str.replace('famíliasequipamentosmateriais', 'equipamentos')
bid_top['DS_OBJETO'] = bid_top['DS_OBJETO'].str.replace('equipamentosmateriais', 'equipamentos')
bid_top['DS_OBJETO'] = bid_top['DS_OBJETO'].str.replace('materiais', 'equipamentos')
bid_top['DS_OBJETO'] = bid_top['DS_OBJETO'].str.replace('medicohospitalaresenfermagem', 'hospitalares')
bid_top['DS_OBJETO'] = bid_top['DS_OBJETO'].str.replace('equipamentos equipamentos', 'equipamentos')
bid_top['DS_OBJETO'] = bid_top['DS_OBJETO'].str.replace('pconstrucao', 'construcao')
item_top['DS_ITEM'] = item_top['DS_ITEM'].str.replace('ctinta', 'tinta')
# Delete variables to free memory
del bid, item, po_top_costs, po_top_costs_bid, df, sf


In [173]:
# Extract n-grams
top_n_grams = extract_ngrams(corpus=item_top['DS_ITEM'],
                                n=2, top_n=10) 
n_gram, total = map(list,zip(*top_n_grams))

df = pd.DataFrame({'Items' : n_gram,
                    'Total' : total})

fig = df.hvplot.barh(x='Items', y='Total', flip_yaxis=True, width=900,
                    title='Figure 7 - Top 10 Items and Services by Mostly Costly Public Organs (2016-219)')
hvplot.save(fig, FIG_PATH+'fig-7.png')
fig

:Bars   [Items]   (Total)

In [174]:

top_n_grams = extract_ngrams(corpus=bid_top['DS_OBJETO'],
                                n=2, top_n=10)
n_gram, total = map(list,zip(*top_n_grams))

df = pd.DataFrame({'Description' : n_gram,
                    'Total' : total})

fig = df.hvplot.barh(x='Description', y='Total', flip_yaxis=True, width=900,
                    title='Figure 8 - Top 10 Bid Types by Mostly Costly Public Organs (2016-219)')
hvplot.save(fig, FIG_PATH+'fig-8.png')
fig

:Bars   [Description]   (Total)

# 6. Anomaly Detection Model

In [175]:
bid_top.drop(['CD_ORGAO'], axis=1, inplace=True)
#item_top.drop(['NM_ORGAO'], axis=1)

,NM_ORGAO,NR_LICITACAO,ANO_LICITACAO,DS_OBJETO
6851,CONS. PÚBL. SAN. BÁSICO DA BACIA HIDROG. DO RI...,10.0,2016,roteador wireless mbps capacidade dispositivos...
8618,CONS. PÚBL. SAN. BÁSICO DA BACIA HIDROG. DO RI...,34.0,2016,aparelho telefônico fixocom fio modo discagem ...
9566,CONS. PÚBL. SAN. BÁSICO DA BACIA HIDROG. DO RI...,11.0,2016,chaveador dlink kvm portas teclado vídeo cabo ...
12337,CIA. RIOGRANDENSE DE SANEAMENTO,190.0,2016,multifuncionais duplex tecnologia impressão la...
12400,PM DE PORTO ALEGRE,414.0,2016,preço contratação empresa especializada vigilâ...
...,...,...,...,...
82049,CIA. RIOGRANDENSE DE SANEAMENTO,108.0,2019,bucha bronze
82050,CIA. RIOGRANDENSE DE SANEAMENTO,192.0,2019,toner kyocera toner kyocera preto toner kyocer...
82096,CENTRAL DE LICITAÇÕES DO ESTADO,184.0,2019,equipamentospecasacessorios pagriculturapecuar...
82097,CENTRAL DE LICITAÇÕES DO ESTADO,190.0,2019,famíliasequipamentos gráficos similares


In [190]:
def join_top_data(bid_t, item_t, year):
    bid_t = bid_t.query('ANO_LICITACAO == @year')
    item_t = item_t.query('ANO_LICITACAO == @year')
    data = bid_t.merge(item_t, on='NR_LICITACAO')
    return data

df = join_top_data(bid_top, item_top, 2018)
df['NM_ORGAO'].nunique()

4